<a href="https://colab.research.google.com/github/Assis-Mohanty/Micrograd/blob/main/Micrograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class Value:
  def __init__(self,data,_children=(),_op='',label=''):
    self.data = data
    self._prev=set(_children)
    self._op=_op
    self.label=label
    self.grad=0
    self._backward=lambda:None

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self,other):
    other=other if isinstance(other,Value) else Value(other)
    out=Value(self.data+other.data,(self,other),'+')
    def _backward():
      self.grad+=1.0*out.grad
      other.grad+=1.0*out.grad
    out._backward=_backward
    return out

  def __radd__(self,other):
    return self+other

  def __neg__(self):
    return self*-1

  def __sub__(self,other):
    return self+(-other)

  def __mul__(self,other):
    other=other if isinstance(other,Value) else Value(other)
    out=Value(self.data*other.data,(self,other),'*')
    def _backward():
      self.grad+=other.grad*out.grad
      other.grad+=self.grad*out.grad
    out._backward=_backward
    return out

  def __pow__(self,other):
    assert isinstance(other,(int,float))
    out=Value(self.data**other,(self,),f'**{other}')
    def _backward():
      self.grad += (other *self.data**(other-1))*out.grad
    out._backward=_backward
    return out

  def __rmul__(self,other):
    return self*other

  def __truediv__(self,other):
    return self*other**-1

  def tanh(self):
    z=self.data
    t=(math.exp(2*z)-1)/(math.exp(2*z)+1)
    out=Value(t,(self,),'tanh')
    def _backward():
      self.grad+=(1-t**2)*out.grad
    out._backward=_backward
    return out

  def exp(self):
    z=self.data
    e=math.exp(z)
    out=Value(e,(self,),'exp')
    def _backward():
      self.grad+=out*out.grad
    out._backward=_backward
    return out

  def backward(self):

    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [ ]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right

  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [ ]:
import torch

In [ ]:
x1=torch.Tensor([2.0]).double()
x2=torch.Tensor([0.0]).double()

In [ ]:
x1 = torch.Tensor([2.0]).double() ; x1.requires_grad = True
x2 = torch. Tensor ([0.0]).double() ; x2.requires_grad = True
w1 = torch. Tensor([-3.0]) .double() ; w1.requires_grad = True
w2 = torch. Tensor([1.0]) .double() ; w2.requires_grad = True
b = torch. Tensor ( [6.8813735870195432]) .double() ; b.requires_grad = True
n= x1*w1 + x2*w2 + b
o = torch. tanh(n)
print(o.data. item())
o.backward()
print('-—"')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

0.7071066904050358
-—"
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
w1 1.0000002567688737


In [ ]:
import random
class Neuron:
  def __init__(self,nin):
    self.w=[Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b=Value(random.uniform(-1,1))
  def __call__(self,x):
    act=sum((wi*xi for wi,xi  in zip(self.w,x)),self.b)
    out=act.tanh()
    return out
  def parameters(self):
    return self.w+[self.b]
class Layer:
  def __init__(self,nin,nout):
    self.neurons=[Neuron(nin) for _ in range(nout)]
  def __call__(self,x):
    outs=[n(x) for n in self.neurons]
    return outs[0] if len(outs)==1 else outs
  def parameters(self):
    params=[]
    for neuron in self.neurons:
      ps=neuron.parameters()
      params.extend(ps)
    return params

class MLP:
  def __init__(self,nin,nouts):
    sz=[nin]+nouts
    self.layers=[Layer(sz[i],sz[i+1]) for  i in range(len(nouts))]

  def __call__(self,x):
    for layer in self.layers:
      x=layer(x)
    return x

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [ ]:
x=[2.0,2.4,1.3]
n=MLP(3,[5,5,1])
n(x)

Value(data=0.8308343924013168)

In [ ]:
n.parameters()

[Value(data=0.5913690109615413),
 Value(data=0.3919393139003311),
 Value(data=0.9996004451597367),
 Value(data=0.745790233357565),
 Value(data=0.8696597251725988),
 Value(data=0.13791795163495402),
 Value(data=-0.14785896304575274),
 Value(data=-0.23680319540467498),
 Value(data=0.8768586672089129),
 Value(data=-0.9696708754192989),
 Value(data=-0.8216356169306136),
 Value(data=-0.28963021106888465),
 Value(data=0.20987928001910028),
 Value(data=0.48719886175497606),
 Value(data=-0.19848272151900725),
 Value(data=-0.6570308038182018),
 Value(data=0.7012754540809263),
 Value(data=0.34705884663632647),
 Value(data=-0.17770101388259252),
 Value(data=-0.48323945037577576),
 Value(data=-0.9525379887869134),
 Value(data=0.5466194015138772),
 Value(data=-0.5944603534619379),
 Value(data=-0.20903825179070012),
 Value(data=0.5252877898656296),
 Value(data=-0.8724080104831411),
 Value(data=-0.6241531291202922),
 Value(data=-0.9780092058173866),
 Value(data=-0.9173803016998103),
 Value(data=-0.23

In [ ]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0]

In [ ]:
for k in range(20):

  # forward pass
  ypred = [n(x) for x in xs]
  loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

  # backward pass
  for p in n.parameters():
    p.grad = 0.0
  loss.backward()

  # update
  for p in n.parameters():
    p.data += -0.1 * p.grad

  print(k, loss.data)

0 2.832223267605169
1 2.832223267605169
2 2.8322232676051695
3 2.8322232676051695
4 2.8322232676051695
5 2.832223267605169
6 2.832223267605169
7 2.832223267605169
8 2.832223267605169
9 2.8322232676051695
10 2.832223267605169
11 2.832223267605169
12 2.832223267605169
13 2.8322232676051695
14 2.8322232676051695
15 2.832223267605169
16 2.832223267605169
17 2.832223267605169
18 2.8322232676051695
19 2.832223267605169


In [ ]:
ypred

[Value(data=0.42587684742471144),
 Value(data=-0.18473167523552644),
 Value(data=-0.2003898151117779),
 Value(data=-0.09479084036218231)]